In [118]:
import pandas as pd
import re
import requests
import bs4

## Читаем файл с предоставленными отзывами 

In [119]:
fname_test = 'test.csv'

In [120]:
fo = open(fname_test, 'r')
data = fo.readlines()
fo.close()

In [121]:
s=0
for i in data:
    print s, i
    s +=1

0 <review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.

1 Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.

2 </review>

3 

т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.

5 работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас 

# Сбор отзывов

In [50]:
#http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=71
#    http://www.e-katalog.ru/list/122/

In [164]:
req = requests.get('http://www.e-katalog.ru/list/122/')
print req

<Response [200]>


In [167]:
print req.text


<!DOCTYPE html><html><head><meta name="viewport" content="width=device-width" data-no-mobile><title>Телефоны на E-katalog.ru > купить мобильный телефон — цены интернет-магазинов России - в Москве, Санкт-Петербурге</title><meta name="keywords" content="Мобильные телефоны, цены, где купить, рейтинг, интернет-магазины, магазины, отзывы, описания"><meta name="description" content="Где купить телефоны по лучшей цене? >>> E-Katalog.ru поможет подобрать! Более 1000 магазинов ✔ Сравнение цен и характеристики"><link rel="alternate" hreflang="ru-ru" href="http://www.e-katalog.ru/list/122/"><link rel="alternate" hreflang="ru-ua" href="http://ek.ua/list/122/"><link href="https://plus.google.com/108552748432756686461" rel="publisher">
<link rel="stylesheet" href="http://www.e-katalog.ru/css/ek-template.min.css?v=v1.10.5">
<link rel="stylesheet" href="http://www.e-katalog.ru/css/ek-list.css?v=v1.10.5">
<link rel="stylesheet" href="http://www.e-katalog.ru/css/prettyPhoto.css?v=v1.10.5">
<script>var 

In [296]:
#Пример ссылки на страницу с отзывами
#http://www.e-katalog.ru/mtools/dot_output/mui_review.php?idg_=756614&amp;p_start_=26&amp;p_end_=35

In [176]:
# Пример карточки товара
y[0]

<div class="ib model-short-links no-mobile"><a class="ib h no-mobile" href="javascript:void(0)" onclick='id_good = 756614; return $.prettyPhoto.open_pg("_3d_8584", "", "");'><u>3D</u></a><a class="ib" href="#" link="/review/samsung-galaxy-s7-32gb/"><u>\u041e\u0442\u0437\u044b\u0432\u044b</u>\xa0<sub>65</sub></a><a class="ib no-mobile" href="#" link="/photo/samsung-galaxy-s7-32gb/" onclick="id_good = 756614;  $.prettyPhoto.type='video'; return $.prettyPhoto.open_pg('', 'Samsung Galaxy S7 32GB', '');"><u>\u0412\u0438\u0434\u0435\u043e</u>\xa0<sub>12</sub></a><a class="ib no-mobile" href="#" link="/photo/samsung-galaxy-s7-32gb/" onclick="id_good = 756614; return $.prettyPhoto.open_pg('/jpg_zoom1/756614.jpg', 'Samsung Galaxy S7 32GB', '');"><u>\u0424\u043e\u0442\u043e</u>\xa0<sub>8</sub></a><div class="ib quest-arch"><a class="ib" href="#" link="/questions/samsung-galaxy-s7-32gb/"><u>\u041e\u0431\u0441\u0443\u0436\u0434\u0435\u043d\u0438\u0435</u></a><sub>9</sub></div><a class="ib no-mobil

### Скрипт для получения ссылок на первые страницы отзывов для моделей

In [60]:
%%writefile parse_ekatalog_links.py
import requests
import bs4
from multiprocessing import Pool
import codecs
import re

def parse_page(url):
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    links = ['http://www.e-katalog.ru'+x.get('link') for x in parser.findAll('a', attrs={'link':re.compile('/review')})]
    #x = parser.findAll('div', attrs={'class':'text'})
    return links

p = Pool(10)
url_list = ['http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=' + str(n) for n in range(1, 72)]
    
if __name__ == '__main__':    
    map_results = p.map(parse_page, url_list)
    reduce_results = reduce(lambda x,y: x + y, map_results)
    with codecs.open('parsing_link_res.txt', 'w', 'utf-8') as output_file:
        print >> output_file, u'\n'.join(reduce_results)

Writing parse_ekatalog_links.py


### Скрипт для получения ссылок на результаты работы скрипта model.php с отзывами на модели в сыром виде

Вариант, формирующий файл для запуска из коммандной строки

In [234]:
%%writefile parse_ekatalog_links.py
import requests
import bs4
from multiprocessing import Pool
import codecs
import re

def parse_model_card(model): # type(model) - bs4 
    #print model
    err = ''
    #model_id_str = model.find('a', attrs={'class': 'h'})
    re_template = "(?<=id_good = ).+?(?=;)"
    model_id_r = re.search(re_template, str(model)) 
    if model_id_r:
        model_id = model_id_r.group(0)
    else:
        error = model.find('a', attrs={'link':re.compile('review')})
        if error:
            err = error.get('link')
            #print "Ошибка:", err
        else:
            err = "000"
            #print "Ошибка:", model
    
    model_rev_count_str = model.find('a', attrs={'link':re.compile('review')})
    if model_rev_count_str:
        model_rev_count = int(model_rev_count_str.find('sub').text)+20
    else: 
        err = model
        model_rev_count = 0
        
    if err == '':
        link = 'http://www.e-katalog.ru/mtools/dot_output/mui_review.php?idg_=' \
        +model_id+'&p_start_=1&p_end_='+str(model_rev_count)
    else: 
        link = ''
    return link, err

def parse_page2(url):   
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    models = parser.findAll('div', attrs={'class': 'model-short-links'})
    links = [parse_model_card(model) for model in models]
    return links

p = Pool(10)
url_list = ['http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=' + str(n) for n in range(1, 73)]

if __name__ == '__main__':    
    map_results = p.map(parse_page2, url_list)
    reduce_results = reduce(lambda x,y: x + y, map_results)
    with codecs.open('parsing_link_res.txt', 'w', 'utf-8') as output_file:
        print >> output_file, u'\n'.join(reduce_results)

Overwriting parse_ekatalog_links.py


Вариант для запуска из ноутбука - для 73 страничек удобнее пользоваться им

In [280]:
def parse_model_card(model): # type(model) - bs4 
    #print model
    err = ''
    #model_id_str = model.find('a', attrs={'class': 'h'})
    re_template = "(?<=id_good = ).+?(?=;)"
    model_id_r = re.search(re_template, str(model)) 
    if model_id_r:
        model_id = model_id_r.group(0)
    else:
        error = model.find('a', attrs={'link':re.compile('review')})
        if error:
            err = error.get('link')
            print "Ошибка:", err
        else:
            err = "000"
            print "Ошибка:", model
    
    model_rev_count_str = model.find('a', attrs={'link':re.compile('review')})
    if model_rev_count_str:
        model_rev_count = int(model_rev_count_str.find('sub').text)+20
    else: 
        model_rev_count = 0
        
    if err == '':
        link = 'http://www.e-katalog.ru/mtools/dot_output/mui_review.php?idg_=' \
        +model_id+'&p_start_=1&p_end_='+str(model_rev_count)
    else: 
        link = ''
    return link, err

def parse_page2(url):  
    print url
    links = []
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    models = parser.findAll('div', attrs={'class': 'model-short-links'})
    for model in models: 
        links.append(parse_model_card(model))
        #print "___"
    return links

url_list = ['http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=' + str(n) for n in range(1, 73)]

In [281]:
results = []
for link in url_list:
    results = results + parse_page2(link)

http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=1
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=2
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=3
Ошибка: /review/nokia-n76/
Ошибка: /review/nokia-6131/
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=4
Ошибка: /review/nokia-6555/
Ошибка: <div class="ib model-short-links no-mobile"><a class="ib no-mobile" href="#" link="/prices/oppo-f5/"><u>Где купить</u> <sub id="list_wb_1232457">257</sub></a></div>
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=5
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=6
Ошибка: <div class="ib model-short-links no-mobile"><a class="ib no-mobile" href="#" link="/prices/oppo-f5-youth/"><u>Где купить</u> <sub id="list_wb_1259042">93</sub></a></div>
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=7
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=8
http://www.e-katalog.ru/ek-list.php?katalog_=122&page_=9
http://www.e-katalog.ru/ek-list.php?katalog_=122&page

In [290]:
df_res

,review,error
0,http://www.e-katalog.ru/mtools/dot_output/mui_...,
1,http://www.e-katalog.ru/mtools/dot_output/mui_...,
2,http://www.e-katalog.ru/mtools/dot_output/mui_...,
3,http://www.e-katalog.ru/mtools/dot_output/mui_...,
4,http://www.e-katalog.ru/mtools/dot_output/mui_...,
5,http://www.e-katalog.ru/mtools/dot_output/mui_...,
6,http://www.e-katalog.ru/mtools/dot_output/mui_...,
7,http://www.e-katalog.ru/mtools/dot_output/mui_...,
8,http://www.e-katalog.ru/mtools/dot_output/mui_...,
9,http://www.e-katalog.ru/mtools/dot_output/mui_...,


In [289]:
df_res = pd.DataFrame(results, columns = ['review', 'error'])

In [297]:
# На всякий случай записываем результат прасинга в файл
df_res.to_csv('out.csv', sep='\t')

In [308]:
review_page_links = list(df_res['review'][df_res['error'] == ""])

In [7]:
len(review_page_links)

1609

In [8]:
df_res

,review,error
,review,error
0,http://www.e-katalog.ru/mtools/dot_output/mui_...,
1,http://www.e-katalog.ru/mtools/dot_output/mui_...,
2,http://www.e-katalog.ru/mtools/dot_output/mui_...,
3,http://www.e-katalog.ru/mtools/dot_output/mui_...,
4,http://www.e-katalog.ru/mtools/dot_output/mui_...,
5,http://www.e-katalog.ru/mtools/dot_output/mui_...,
6,http://www.e-katalog.ru/mtools/dot_output/mui_...,
7,http://www.e-katalog.ru/mtools/dot_output/mui_...,
8,http://www.e-katalog.ru/mtools/dot_output/mui_...,


### Считываем и сохраняем файлы по ссылкам

In [12]:
# считываем сохраненный список ссылок на страницы с отзывами
df_res = pd.read_csv('out.csv', sep='\t',  names = ['review', 'error'], na_filter = False)
review_page_links = list(df_res['review'][df_res['error'] == ""])

In [9]:
# выделяем id модели из ссылки
def get_id_fromlink(link):
    re_template = "(?<=idg_=).+?(?=&p)"
    model_id = re.search(re_template, link) 
    if model_id:
        return model_id.group(0)
    else: return ''

In [19]:
def save_file(filename, text_to_save):
    file_obj = open(filename, 'w')   
    file_obj.write(text_to_save)
    file_obj.close()
    
def save_review_page(link):
    model_id = get_id_fromlink(link)
    if model_id != '':
        temp_filename = 'tempdir/'+model_id+'.txt'
    req = requests.get(link)
    if req: 
        body = req.text.encode('utf-8')
    if body and temp_filename:
        save_file(temp_filename, body)
    else: print link

In [28]:
for link in review_page_links:
    save_review_page(link)

### Первоначальный вариант парсинга ссылок на первую страницу отзывов
От него отказались в пользу общей выдачи

In [15]:
parser = bs4.BeautifulSoup(req.text, 'lxml')

In [53]:
print type(parser)
print parser

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html>
<html><head><meta content="width=device-width" data-no-mobile="" name="viewport"/><title>Телефоны на E-katalog.ru &gt; купить мобильный телефон — цены интернет-магазинов России - в Москве, Санкт-Петербурге</title><meta content="Мобильные телефоны, цены, где купить, рейтинг, интернет-магазины, магазины, отзывы, описания" name="keywords"/><meta content="Где купить телефоны по лучшей цене? &gt;&gt;&gt; E-Katalog.ru поможет подобрать! Более 1000 магазинов ✔ Сравнение цен и характеристики" name="description"/><link href="http://www.e-katalog.ru/list/122/" hreflang="ru-ru" rel="alternate"/><link href="http://ek.ua/list/122/" hreflang="ru-ua" rel="alternate"/><link href="https://plus.google.com/108552748432756686461" rel="publisher"/>
<link href="http://www.e-katalog.ru/css/ek-template.min.css?v=v1.10.5" rel="stylesheet"/>
<link href="http://www.e-katalog.ru/css/ek-list.css?v=v1.10.5" rel="stylesheet"/>
<link href="http://www.e-katalog.ru/css/pretty

In [17]:
x = parser.findAll('div', attrs={'class':'model-short-links'})
print type(x)


<class 'bs4.element.ResultSet'>


In [55]:
parser.findAll('a', attrs={'link':re.compile('/review')})[0].get('link')

'/review/samsung-galaxy-s7-32gb/'

In [57]:
links = ['http://www.e-katalog.ru'+x.get('link') for x in parser.findAll('a', attrs={'link':re.compile('/review')})]

In [63]:
fo = open('parsing_link_res.txt', 'r')
data = fo.readlines()
fo.close()

In [65]:
len(data)

1187

In [67]:
data[265:270]

['http://www.e-katalog.ru/review/lg-g4-32gb-duos/\n',
 'http://www.e-katalog.ru/review/motorola-razr2-v8/\n',
 'http://www.e-katalog.ru/review/nokia-3710-fold/\n',
 'http://www.e-katalog.ru/review/samsung-galaxy-c9-pro/\n',
 'http://www.e-katalog.ru/review/htc-one-x9-dual-sim/\n']

In [ ]:
http://www.e-katalog.ru/mtools/dot_output/mui_review.php?idg_=588881&
    p_start_=1&p_end_=106&callback=jQuery220021198801772120046_1512850209047&_=1512850209052

### Парсинг страницы с отзывами

In [36]:
import glob
import os

In [8]:
minus = 'review-m.gif'  #- недостатки
plus = 'review-p.gif' #- достоинства
grad1 = 'review-smile-1.gif' #- все плохо
grad2 = 'review-smile-2.gif' #- так себе
grad3 = 'review-smile-3.gif' #- нормально
grad4 = 'review-smile-4.gif' #- отлично

In [25]:
def read_file(filename):
    fo = open(filename, 'r')
    data = fo.read()
    fo.close()
    return data
# Делаем beautifullSoup из ссылки
def parse_review_page(link):
    text = read_file(link)
    parser = bs4.BeautifulSoup(text, 'lxml')
    return parser

In [27]:
l = parse_review_page('tempdir/932060.txt')

In [319]:
#u = 'file:///Users/elnafrolova/Yandex.Disk.localized/coursera/final_project/week6_kaggle_100/mui_review.html'
u = parse_review_page(review_page_links[1])[1]

In [30]:
l

<html><body><p>('<a name="\\'fap5uhdopk1\\'"></a></p><div class='\\"review\\"' id="\\'item-mod-fap5uhdopk1\\'"><table class="\\'review-table\\'"><tr><td class="\\'review-helpful\\'"><div class="\\'helpful\\'" id="\\'yn-fap5uhdopk1\\'"></div><div class='\\"clr\\"'></div><div class='\\"op-logo-ya\\"' title='\\"\u041e\u0442\u0437\u044b\u0432'></div><script data-no-mobile="">vote_data.push([\\'\\', \\'Reviews\\', \\'fap5uhdopk1\\', \\'6\\', \\'0\\', \\'2qc8gxcpjb\\', \\'http://www.e-katalog.ru/mtools/\\',\\'color-bar\\']);</script></td><td class="\\'review-td\\'"><table width="\\'100%\\'"><tr valign="top"><td><div class="\\'review-title\\'"><span property='\\"v:summary\\"'>\u043e\u0442\u043b\u0438\u0447\u043d\u0430\u044f \u043c\u043e\u0434\u0435\u043b\u044c</span><img alt="\\'\\'" src="\\'/img/review-smile-4.gif\\'"/></div></td><td align="\\'right\\'" nowrap=""><div class="\\'review-date\\'">\xa0<span>29 \u043d\u043e\u044f\u0431\u0440\u044f 2016</span>\xa0<a href="\\'#\\'" link="\\'/rw/xia

In [43]:
# Разделяем отзывы со страницы
def parse_review_list(review_page):
    return review_page.findAll('td', attrs={'class':re.compile('review-td')})

In [332]:
list_1 = parse_review_list(u)

In [44]:
# Для каждого отзыва собираем части информации
def parse_review(review):
    grad = 0
    title, comment, plus, minus = '','','',''
    y_smile = review.find('img', attrs={'src':re.compile('review-smile')})    
    y_title = review.find('div', attrs={'class':re.compile('review-title')})
    y_comment = review.find('div', attrs={'class':re.compile('review-comment')})
    y_plus = review.find('div', attrs={'class':re.compile('review-plus')})
    y_minus = review.find('div', attrs={'class':re.compile('review-minus')})
    if y_smile:
        re_template = "(?<=review-smile-).+(?=.gif)"
        y_grad = re.search(re_template, y_smile.get('src'))
        grad = int(y_grad.group(0))
    if y_title:
        title = y_title.text
    if y_comment:
        comment = y_comment.text
    if y_plus:
        plus = y_plus.text
    if y_minus:
        minus = y_minus.text
    
    return grad, title, comment, plus, minus
    #return y_smile, y_title.text, y_comment[0].text, y_plus[0].text, y_minus[0].text

In [66]:
files = glob.glob(os.path.join('tempdir/', '*.txt'))
titles = []
comments = []
pluses = []
minuses = []
for page in files:
    parser = parse_review_page(page)
    list1 = parse_review_list(parser) 
    print len(list1)
    for i in list1:
        i,t,c,p,m = parse_review(i)
        titles.append((i,t))
        comments.append((i,c))
        pluses.append(p)
        minuses.append(m)

3
3
81
58
35
0
39
80
5
7
5
0
11
0
47
10
0
3
0
21
53
61
59
0
38
41
0
48
16
59
37
0
0
9
46
1
25
1
10
41
158
0
68
63
29
2
1
0
2
32
0
60
77
1
17
0
0
0
12
4
6
1
9
1
17
11
54
70
60
44
4
7
0
98
0
0
3
53
0
0
6
0
1
1
0
34
0
48
0
38
39
6
7
1
35
0
1
38
50
0
7
0
0
8
0
37
98
0
42
24
0
0
1
6
2
2
0
0
0
0
50
0
0
0
32
0
2
15
0
2
11
2
79
93
10
0
27
8
3
61
155
17
0
16
10
10
0
1
176
43
14
40
0
1
42
0
0
0
3
0
15
0
37
1
30
63
3
4
7
82
11
31
0
7
13
0
1
19
0
0
9
0
5
2
0
58
0
52
0
0
0
15
15
8
0
40
91
4
47
1
0
46
60
47
4
9
0
37
0
81
52
0
7
8
0
0
85
5
18
15
9
85
13
0
0
1
24
16
0
0
39
0
63
12
125
6
3
10
2
0
81
3
9
3
35
0
31
47
0
0
10
0
1
50
14
13
0
0
0
28
2
43
0
1
48
0
29
12
48
5
12
1
55
0
1
0
1
95
48
3
118
85
0
0
0
2
6
71
1
32
0
8
2
7
3
16
8
0
1
22
4
1
38
11
27
24
0
47
1
1
0
0
0
0
2
72
0
16
110
16
98
14
0
14
63
0
1
0
22
7
0
53
0
0
2
0
46
4
71
58
5
74
33
0
18
17
0
56
0
10
0
0
0
4
0
9
0
76
25
1
108
0
55
0
37
6
1
45
46
72
15
40
38
87
10
11
4
0
0
1
16
46
13
5
0
4
0
41
1
30
0
30
58
19
0
6
47
37
0
0
9
69
58
35
55
15
1

In [68]:
print len(comments), len(pluses)

33740 33740


In [114]:
df_titles = pd.DataFrame(titles, columns = ['grade', 'title'])
df_comments = pd.DataFrame(comments, columns = ['grade', 'comment'])

In [116]:
df_comments['grade'].value_counts()

4    17825
3     7811
0     2801
1     2790
2     2513
Name: grade, dtype: int64

Запишем полученные результаты в файлы на будущее:

In [75]:
save_file('minus.txt', u'\nminus\n'.join(minuses).encode('utf-8'))
save_file('plus.txt', u'\nplus\n'.join(pluses).encode('utf-8'))

In [109]:
df_titles.to_csv('titles.csv', sep='\t', encoding = 'utf-8')
df_comments.to_csv('comments.csv', sep='\t', encoding = 'utf-8')

 ### Обучение модели - в новом файле
 ### Парсинг тестовых отзывов

In [122]:
s=0
for i in data:
    print s, i
    s +=1

0 <review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.

1 Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.

2 </review>

3 

т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.

5 работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас 

In [157]:
fo = open(fname_test, 'r')
data1 = fo.read()
fo.close()

In [158]:
s=0
for i in data:
    print i
    s +=1

<review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.

Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.

</review>



т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.

работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас , но на со

In [167]:
reviews = ' '.join(data).replace('<review>', '').split('</review>')

In [168]:
for i in reviews:
    print i

Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
 Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.
 

 
т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.
 работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас , но на солнце сохраняет чи

In [220]:
test = [x.replace('\n', ' ').replace('\r', ' ').strip() for x in reviews[0:100]]

In [221]:
v = pd.DataFrame(test)

In [222]:
v.to_csv('test1.csv', sep='\t')

In [223]:
v

,0
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях К сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."
5,- Удобная Клавиатура и русская раскладка - 2 с...
6,Супер телефон! 1.QWERTY!!! Это самый лучший ег...
7,- толщина (помещается даже в брюки) - аккумуля...
8,Аккумулятор ужасен! Хватает буквально на неско...
9,1 удобный.клавеатура просто класс быстро пишеш...
